# IMPORTING LIBRARIES

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',-1)

# IMPORTING AND FILTERING DATASET

In [2]:
df = pd.read_csv('C:/Users/Shree/Desktop/onceAgain/csv_files/T20_AROUND_THE_WORLD/All_Matches_CSVs/IPL.csv')

In [3]:
if type(df.ball[0])==type('str'):
    df = df.drop('ball', axis=1)
else:
    pass

if ('overs') in df.columns:
    df = df.rename(columns={'overs':'ball'})
else:
    pass 

df = df.sort_values(['match_id','innings','ball'],ascending=[True,True,True])

df.reset_index(inplace=True,drop=True)

overs = df.copy()
overs['over_no'] = ''
for i in range(len(overs)):
    ball_no = overs['ball'][i]
    over = int(str(ball_no).split('.')[0])
    overs['over_no'][i] = over
df['over_no'] = overs['over_no'] + 1

total0 = pd.DataFrame(df.groupby(['match_id','innings'])[(['runs_off_bat']+['extras'])].sum()).rename(columns={'runs_off_bat':'A','extras':'B'}).reset_index()
total0['inn_total'] = total0.apply(lambda x : x['A'] + x['B'], axis=1)

total1 = pd.DataFrame(df[df.innings==1].groupby(['match_id','innings'])[(['runs_off_bat']+['extras'])].sum()).rename(columns={'runs_off_bat':'A','extras':'B'}).reset_index()
total1['inn1_total'] = total1.apply(lambda x : x['A'] + x['B'], axis=1)

total = pd.merge(total0,total1,on=['match_id'])

total = total.drop(['A_x','B_x','innings_y','A_y','B_y'],axis=1).rename(columns={'innings_x':'innings'})

df = pd.merge(df,total,on=['match_id','innings'])

# IMPORTANT FUNCTIONS FOR DATA MAKING

In [4]:
def phase(over_no):
    if over_no <= 6:
        return 'Powerplay'
    elif over_no <= 15:
        return 'Middle'
    else:
        return 'Death'
if ('phase') not in df.columns:
    df['phase'] = df['over_no'].apply(lambda x: phase(x))
    
def BPB(balls,boundaries):
    if boundaries == 0:
        return balls/1
    else:
        return balls/boundaries
    
def BPD(balls,dismissals):
    if dismissals == 0:
        return balls/1
    else:
        return balls/dismissals
    
def Bowler_WK(player_dismissed,wicket_type):
    if type(player_dismissed)==str:
        if wicket_type not in ['run out','retired hurt', 'obstructing the field', 'retired out']:
            return 1
        else:
            return 0
    else:
        return 0

# INDIVIDUAL FEATURE NAME FOR APPLYING IN FUNCTIONS:

In [5]:
recent_teams = ['Royal Challengers Bangalore', 'Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals', 'Mumbai Indians',
     'Sunrisers Hyderabad', 'Delhi Capitals', 'Punjab Kings', 'Gujarat Titans', 'Lucknow Super Giants']

# batters = [A Ashish Reddy : A Badoni : A Chandila : A Chopra : A Choudhary : A Dananjaya : A Flintoff : A Kumble : A Manohar : A Mishra : A Mithun : A Mukund : A Nehra : A Nortje : A Singh : A Symonds : A Tomar : A Uniyal : A Zampa : AA Bilakhia : AA Chavan : AA Jhunjhunwala : AA Noffke : AB Agarkar : AB Barath : AB Dinda : AB McDonald : AB de Villiers : AC Blizzard : AC Gilchrist : AC Thomas : AC Voges : AD Hales : AD Mascarenhas : AD Mathews : AD Nath : AD Russell : AF Milne : AG Murtaza : AG Paunikar : AJ Finch : AJ Hosein : AJ Turner : AJ Tye : AK Markram : AL Menaria : AM Nayar : AM Rahane : AN Ahmed : AN Ghosh : AP Dole : AP Majumdar : AP Tare : AR Bawne : AR Patel : AS Joseph : AS Rajpoot : AS Raut : AS Roy : AS Yadav : AT Carey : AT Rayudu : AU Rashid : AUK Pathan : Abdul Basith : Abdul Samad : Abdur Razzak : Abhishek Sharma : Abishek Porel : Akash Deep : Aman Hakim Khan : Anirudh Singh : Ankit Sharma : Ankit Soni : Anmolpreet Singh : Anuj Rawat : Anureet Singh : Arjun Tendulkar : Arshad Khan (2) : Arshdeep Singh : Atharva Taide : Avesh Khan : Azhar Mahmood : B Akhil : B Chipli : B Indrajith : B Kumar : B Laughlin : B Lee : B Sai Sudharsan : B Stanlake : B Sumanth : BA Bhatt : BA Stokes : BAW Mendis : BB McCullum : BB Samantray : BB Sran : BCJ Cutting : BE Hendricks : BJ Haddin : BJ Hodge : BJ Rohrer : BMAJ Mendis : BR Dunk : Basil Thampi : Bipul Sharma : C Green : C Madan : C Munro : C Nanda : C Sakariya : C de Grandhomme : CA Ingram : CA Lynn : CA Pujara : CH Gayle : CH Morris : CJ Anderson : CJ Ferguson : CJ Jordan : CJ McKay : CK Kapugedera : CK Langeveldt : CL White : CM Gautam : CR Brathwaite : CR Woakes : CRD Fernando : CV Varun : D Brevis : D Kalyankrishna : D Padikkal : D Pretorius : D Salunkhe : D Wiese : D du Preez : DA Miller : DA Warner : DAJ Bracewell : DB Das : DB Ravi Teja : DE Bollinger : DG Nalkande : DH Yagnik : DJ Bravo : DJ Harris : DJ Hooda : DJ Hussey : DJ Jacobs : DJ Malan : DJ Mitchell : DJ Muthuswami : DJ Thornely : DJ Willey : DJG Sammy : DJM Short : DL Chahar : DL Vettori : DM Bravo : DNT Zoysa : DP Conway : DP Nannes : DP Vijaykumar : DPMD Jayawardene : DR Martyn : DR Sams : DR Shorey : DR Smith : DS Kulkarni : DS Lehmann : DT Christian : DT Patil : DW Steyn : Dhruv Jurel : E Lewis : EJG Morgan : ER Dwivedi : F Behardien : F du Plessis : FA Allen : FH Edwards : FY Fazal : Fazalhaq Farooqi : G Gambhir : GB Hogg : GC Smith : GC Viljoen : GD McGrath : GD Phillips : GH Vihari : GHS Garton : GJ Bailey : GJ Maxwell : GR Napier : Gurkeerat Singh : H Das : H Klaasen : HC Brook : HE van der Dussen : HF Gurney : HH Gibbs : HH Pandya : HM Amla : HR Shokeen : HV Patel : Harbhajan Singh : Harmeet Singh : Harpreet Brar : Harpreet Singh : Harshit Rana : I Malhotra : I Sharma : I Udana : IC Pandey : IK Pathan : IR Jaggi : IS Sodhi : Imran Tahir : Iqbal Abdulla : Ishan Kishan : J Arunkumar : J Botha : J Suchith : J Syed Mohammad : J Theron : J Yadav : JA Morkel : JA Richardson : JC Archer : JC Buttler : JD Ryder : JD Unadkat : JDP Oram : JDS Neesham : JE Root : JE Taylor : JEC Franklin : JH Kallis : JJ Bumrah : JJ Roy : JJ van der Wath : JL Denly : JL Pattinson : JM Bairstow : JM Kemp : JM Sharma : JO Holder : JP Behrendorff : JP Duminy : JP Faulkner : JPR Scantlebury-Searles : JR Hazlewood : JR Hopes : JR Philippe : Jaskaran Singh : Joginder Sharma : K Goel : K Gowtham : K Kartikeya : K Rabada : K Upadhyay : K Yadav : KA Jamieson : KA Pollard : KAJ Roach : KB Arun Karthik : KC Cariappa : KC Sangakkara : KD Karthik : KH Pandya : KJ Abbott : KK Ahmed : KK Cooper : KK Nair : KL Nagarkoti : KL Rahul : KM Asif : KM Jadhav : KMA Paul : KMDN Kulasekara : KP Appanna : KP Pietersen : KR Mayers : KR Sen : KS Bharat : KS Sharma : KS Williamson : KV Sharma : KW Richardson : Kamran Akmal : Kamran Khan : Karanveer Singh : Kartik Tyagi : Kuldeep Yadav : L Ablish : L Balaji : L Ronchi : LA Carseldine : LA Pomersbach : LE Plunkett : LH Ferguson : LJ Wright : LMP Simmons : LPC Silva : LR Shukla : LRPL Taylor : LS Livingstone : Lalit Yadav : Liton Das : M Ashwin : M Jansen : M Kaif : M Kartik : M Klinger : M Manhas : M Markande : M Morkel : M Muralitharan : M Ntini : M Prasidh Krishna : M Rawat : M Shahrukh Khan : M Theekshana : M Vijay : M Vohra : M de Lange : MA Agarwal : MA Khote : MA Starc : MA Wood : MC Henriques : MC Juneja : MD Mishra : MD Shanaka : MDKJ Perera : MEK Hussey : MF Maharoof : MG Bracewell : MG Johnson : MJ Clarke : MJ Guptill : MJ Lumb : MJ McClenaghan : MJ Santner : MK Lomror : MK Pandey : MK Tiwary : ML Hayden : MM Ali : MM Patel : MM Sharma : MN Samuels : MN van Wyk : MP Stoinis : MR Marsh : MS Bisla : MS Dhoni : MS Gony : MS Wade : MV Boucher : MW Short : Mandeep Singh : Mashrafe Mortaza : Misbah-ul-Haq : Mohammad Ashraful : Mohammad Asif : Mohammad Hafeez : Mohammad Nabi : Mohammed Shami : Mohammed Siraj : Mohit Rathee : Mohsin Khan (2) : Mujeeb Ur Rahman : Mukesh Choudhary : Mukesh Kumar : Mustafizur Rahman : N Jagadeesan : N Pooran : N Rana : N Saini : N Wadhera : NA Saini : ND Doshi : NJ Maddinson : NJ Rimmington : NL McCullum : NLTC Perera : NM Coulter-Nile : NS Naik : NT Ellis : NV Ojha : Naveen-ul-Haq : Niraj Patel : Noor Ahmad : OA Shah : OC McCoy : OF Smith : P Awana : P Chopra : P Dogra : P Dubey : P Kumar : P Negi : P Parameswaran : P Ray Barman : P Sahu : P Simran Singh : PA Patel : PA Reddy : PBB Rajapaksa : PC Valthaty : PD Collingwood : PD Salt : PJ Cummins : PJ Sangwan : PK Garg : PM Sarvesh Kumar : PN Mankad : PP Chawla : PP Ojha : PP Shaw : PR Shah : PSP Handscomb : PV Tambe : PVD Chameera : PWH de Silva : Pankaj Singh : Parvez Rasool : Q de Kock : R Ashwin : R Bhatia : R Bishnoi : R Dhawan : R Dravid : R McLaren : R Ninan : R Parag : R Powell : R Rampaul : R Sanjay Yadav : R Sathish : R Sharma : R Shepherd : R Shukla : R Tewatia : R Vinay Kumar : RA Bawa : RA Jadeja : RA Tripathi : RD Chahar : RD Gaikwad : RE Levi : RE van der Merwe : RG More : RG Sharma : RJ Harris : RJ Peterson : RJ Quiney : RK Bhui : RK Singh : RM Patidar : RN ten Doeschate : RP Meredith : RP Singh : RR Bhatkal : RR Pant : RR Powar : RR Raje : RR Rossouw : RR Sarwan : RS Bopara : RS Gavaskar : RS Sodhi : RT Ponting : RV Gomez : RV Patel : RV Uthappa : Rahmanullah Gurbaz : Ramandeep Singh : Rashid Khan : Rasikh Salam : Ravi Bishnoi : S Anirudha : S Aravind : S Badree : S Badrinath : S Chanderpaul : S Dhawan : S Dube : S Gopal : S Kaul : S Kaushik : S Ladda : S Lamichhane : S Nadeem : S Narwal : S Rana : S Randiv : S Sohal : S Sreesanth : S Sriram : S Tyagi : S Vidyut : SA Abbott : SA Asnodkar : SA Yadav : SB Bangar : SB Jakati : SB Joshi : SB Styris : SB Wagh : SC Ganguly : SD Chitnis : SD Lad : SE Bond : SE Marsh : SE Rutherford : SJ Srivastava : SK Raina : SK Trivedi : SK Warne : SL Malinga : SM Curran : SM Harwood : SM Katich : SM Pollock : SMSM Senanayake : SN Khan : SN Thakur : SO Hetmyer : SP Fleming : SP Goswami : SP Jackson : SP Narine : SPD Smith : SR Tendulkar : SR Watson : SS Cottrell : SS Iyer : SS Prabhudessai : SS Shaikh : SS Tiwary : ST Jayasuriya : STR Binny : SV Samson : SW Billings : SW Tait : Sachin Baby : Salman Butt : Sandeep Sharma : Sanvir Singh : Shahbaz Ahmed : Shahid Afridi : Shakib Al Hasan : Shashank Singh : Shivam Mavi : Shivam Sharma : Shoaib Ahmed : Shoaib Akhtar : Shoaib Malik : Shubman Gill : Sikandar Raza : Simarjeet Singh : Sohail Tanvir : Sunny Gupta : Sunny Singh : Swapnil Singh : T Banton : T Henderson : T Kohli : T Natarajan : T Shamsi : T Stubbs : T Taibu : T Thushara : TA Boult : TD Paine : TG Southee : TH David : TK Curran : TL Seifert : TL Suman : TM Dilshan : TM Head : TM Srivastava : TR Birt : TS Mills : TU Deshpande : Tilak Varma : U Kaul : UA Birla : UBT Chand : UT Khawaja : UT Yadav : Umar Gul : Umran Malik : V Kohli : V Pratap Singh : V Sehwag : V Shankar : VG Arora : VH Zol : VR Aaron : VR Iyer : VRV Singh : VS Malik : VS Yeligati : VVS Laxman : VY Mahesh : Vijaykumar Vyshak : Virat Singh : Vishnu Vinod : Vivrant Sharma : W Jaffer : WA Mota : WD Parnell : WP Saha : WPUJC Vaas : Washington Sundar : X Thalaivan Sargunam : Y Gnaneswara Rao : Y Nagar : Y Prithvi Raj : Y Venugopal Rao : YA Abdulla : YBK Jaiswal : YK Pathan : YS Chahal : YV Dhull : YV Takawale : Yash Dayal : Yashpal Singh : Younis Khan : Yudhvir Singh : Yuvraj Singh : Z Khan]
# bowlers = [A Ashish Reddy : A Badoni : A Chandila : A Choudhary : A Dananjaya : A Flintoff : A Kumble : A Mishra : A Mithun : A Nehra : A Nel : A Nortje : A Singh : A Symonds : A Uniyal : A Zampa : AA Chavan : AA Jhunjhunwala : AA Kazi : AA Noffke : AB Agarkar : AB Dinda : AB McDonald : AC Gilchrist : AC Thomas : AC Voges : AD Mascarenhas : AD Mathews : AD Russell : AF Milne : AG Murtaza : AJ Finch : AJ Hosein : AJ Tye : AK Markram : AL Menaria : AM Nayar : AM Rahane : AM Salvi : AN Ahmed : AP Dole : AR Patel : AS Joseph : AS Rajpoot : AS Raut : AS Roy : AU Rashid : AUK Pathan : Abdul Samad : Abdur Razzak : Abhishek Sharma : Akash Deep : Akash Madhwal : Akash Singh : Aman Hakim Khan : Anand Rajan : Ankit Sharma : Ankit Soni : Anureet Singh : Arjun Tendulkar : Arshad Khan (2) : Arshdeep Singh : Avesh Khan : Azhar Mahmood : B Akhil : B Chipli : B Geeves : B Kumar : B Laughlin : B Lee : B Stanlake : BA Bhatt : BA Stokes : BAW Mendis : BB Sran : BCJ Cutting : BE Hendricks : BJ Hodge : BJ Rohrer : BMAJ Mendis : BW Hilfenhaus : Basil Thampi : Bipul Sharma : C Ganapathy : C Green : C Munro : C Nanda : C Sakariya : C de Grandhomme : CH Gayle : CH Morris : CJ Anderson : CJ Dala : CJ Green : CJ Jordan : CJ McKay : CK Kapugedera : CK Langeveldt : CL White : CR Brathwaite : CR Woakes : CRD Fernando : CV Varun : D Brevis : D Kalyankrishna : D Pretorius : D Salunkhe : D Wiese : D du Preez : DA Warner : DAJ Bracewell : DB Ravi Teja : DE Bollinger : DG Nalkande : DJ Bravo : DJ Harris : DJ Hooda : DJ Hussey : DJ Mitchell : DJ Muthuswami : DJ Thornely : DJ Willey : DJG Sammy : DJM Short : DL Chahar : DL Vettori : DNT Zoysa : DP Nannes : DP Vijaykumar : DR Sams : DR Smith : DS Kulkarni : DT Christian : DW Steyn : Duan Jansen : F du Plessis : FA Allen : FH Edwards : FY Fazal : Fazalhaq Farooqi : GB Hogg : GC Viljoen : GD McGrath : GD Phillips : GH Vihari : GHS Garton : GJ Maxwell : GR Napier : GS Sandhu : Gagandeep Singh : Gurkeerat Singh : Gurnoor Brar : H Sharma : HF Gurney : HH Pandya : HR Shokeen : HV Patel : Harbhajan Singh : Harmeet Singh : Harmeet Singh (2) : Harpreet Brar : Harshit Rana : I Malhotra : I Sharma : I Udana : IC Pandey : IC Porel : IK Pathan : IS Sodhi : Imran Tahir : Iqbal Abdulla : J Botha : J Little : J Suchith : J Syed Mohammad : J Theron : J Yadav : JA Morkel : JA Richardson : JC Archer : JD Ryder : JD Unadkat : JDP Oram : JDS Neesham : JE Root : JE Taylor : JEC Franklin : JH Kallis : JJ Bumrah : JJ van der Wath : JL Pattinson : JM Kemp : JO Holder : JP Behrendorff : JP Duminy : JP Faulkner : JPR Scantlebury-Searles : JR Hazlewood : JR Hopes : JW Hastings : Jalaj S Saxena : Jaskaran Singh : Joginder Sharma : K Goel : K Gowtham : K Kartikeya : K Khejroliya : K Rabada : K Santokie : K Upadhyay : K Yadav : KA Jamieson : KA Pollard : KAJ Roach : KC Cariappa : KH Pandya : KJ Abbott : KK Ahmed : KK Cooper : KL Nagarkoti : KM Asif : KMA Paul : KMDN Kulasekara : KP Appanna : KP Pietersen : KR Mayers : KR Sen : KS Williamson : KV Sharma : KW Richardson : Kamran Khan : Karanveer Singh : Kartik Tyagi : Kuldeep Yadav : L Ablish : L Balaji : L Ngidi : LA Carseldine : LE Plunkett : LH Ferguson : LI Meriwala : LJ Wright : LMP Simmons : LPC Silva : LR Shukla : LRPL Taylor : LS Livingstone : Lalit Yadav : M Ashwin : M Jansen : M Kartik : M Manhas : M Markande : M Morkel : M Muralitharan : M Ntini : M Pathirana : M Prasidh Krishna : M Theekshana : M Vijay : M de Lange : MA Khote : MA Starc : MA Wood : MB Parmar : MC Henriques : MF Maharoof : MG Bracewell : MG Johnson : MG Neser : MJ Clarke : MJ Henry : MJ McClenaghan : MJ Santner : MK Lomror : MK Tiwary : MM Ali : MM Patel : MM Sharma : MN Samuels : MP Stoinis : MR Marsh : MS Gony : MW Short : Mandeep Singh : Mashrafe Mortaza : Mayank Dagar : Mohammad Asif : Mohammad Hafeez : Mohammad Nabi : Mohammed Shami : Mohammed Siraj : Mohit Rathee : Mohsin Khan (2) : Monu Kumar : Mujeeb Ur Rahman : Mukesh Choudhary : Mukesh Kumar : Mustafizur Rahman : N Rana : NA Saini : NB Singh : ND Doshi : NJ Rimmington : NL McCullum : NLTC Perera : NM Coulter-Nile : NT Ellis : Naveen-ul-Haq : Nithish Kumar Reddy : Noor Ahmad : O Thomas : OC McCoy : OF Smith : P Amarnath : P Awana : P Dubey : P Kumar : P Negi : P Parameswaran : P Prasanth : P Ray Barman : P Sahu : P Suyal : PC Valthaty : PD Collingwood : PH Solanki : PJ Cummins : PJ Sangwan : PM Sarvesh Kumar : PP Chawla : PP Ojha : PV Tambe : PVD Chameera : PWH de Silva : Pankaj Singh : Parvez Rasool : R Ashwin : R Bhatia : R Dhawan : R Goyal : R McLaren : R Ninan : R Parag : R Powell : R Rampaul : R Sai Kishore : R Sanjay Yadav : R Sathish : R Sharma : R Shepherd : R Shukla : R Tewatia : R Vinay Kumar : RA Jadeja : RA Shaikh : RA Tripathi : RD Chahar : RE van der Merwe : RG More : RG Sharma : RJ Harris : RJ Peterson : RJW Topley : RN ten Doeschate : RP Meredith : RP Singh : RR Bhatkal : RR Bose : RR Powar : RR Raje : RS Bopara : RS Gavaskar : RS Hangargekar : RV Gomez : RV Patel : RW Price : Ramandeep Singh : Rashid Khan : Rasikh Salam : Ravi Bishnoi : S Aravind : S Badree : S Dhawan : S Dube : S Gopal : S Kaul : S Kaushik : S Ladda : S Lamichhane : S Midhun : S Nadeem : S Narwal : S Rana : S Randiv : S Sandeep Warrier : S Sreesanth : S Sriram : S Tyagi : S Vidyut : SA Abbott : SA Yadav : SB Bangar : SB Jakati : SB Joshi : SB Styris : SB Wagh : SC Ganguly : SC Kuggeleijn : SD Chitnis : SE Bond : SE Rutherford : SJ Srivastava : SK Raina : SK Trivedi : SK Warne : SL Malinga : SM Boland : SM Curran : SM Harwood : SM Pollock : SMSM Senanayake : SN Khan : SN Thakur : SP Narine : SPD Smith : SR Tendulkar : SR Watson : SS Agarwal : SS Cottrell : SS Iyer : SS Mundhe : SS Sarkar : SSB Magala : ST Jayasuriya : STR Binny : SW Tait : Sachin Baby : Sandeep Sharma : Shahbaz Ahmed : Shahid Afridi : Shakib Al Hasan : Shashank Singh : Shivam Mavi : Shivam Sharma : Shoaib Ahmed : Shoaib Akhtar : Shoaib Malik : Sikandar Raza : Simarjeet Singh : Sohail Tanvir : Sunny Gupta : Suyash Sharma : Swapnil Singh : T Henderson : T Natarajan : T Shamsi : T Stubbs : T Thushara : TA Boult : TG Southee : TK Curran : TL Suman : TM Dilshan : TM Head : TP Sudhindra : TS Mills : TU Deshpande : Tejas Baroka : Tilak Varma : UT Yadav : Umar Gul : Umran Malik : V Kohli : V Pratap Singh : V Sehwag : V Shankar : VG Arora : VR Aaron : VR Iyer : VRV Singh : VS Malik : VS Yeligati : VY Mahesh : Vijaykumar Vyshak : Vivrant Sharma : WA Mota : WD Parnell : WPUJC Vaas : Washington Sundar : Y Gnaneswara Rao : Y Nagar : Y Prithvi Raj : Y Venugopal Rao : YA Abdulla : YBK Jaiswal : YK Pathan : YS Chahal : Yash Dayal : Yash Thakur : Yudhvir Singh : Yuvraj Singh : Z Khan]

# BATTING FUNCTION

In [6]:
def Batter_data_fetching(df, team, opposition, phase, a, b, inning, striker):
    
        
    # to get required team:

    if team == 0:
        df = df
    elif team == recent_teams:
        df = df[df.batting_team.isin(recent_teams)].reset_index(drop=True)
    else:
        df = df[df.batting_team == team].reset_index(drop=True)
        
    # to get required opposition:

    if opposition == 0:
        df = df
    elif opposition == recent_teams:
        df = df[df.bowling_team.isin(recent_teams)].reset_index(drop=True)
    else:
        df = df[df.bowling_team == opposition].reset_index(drop=True)
    
    # to get dataset between given phase:
   
    if phase == 0:
        df = df
    else:        
        df = df[df.phase == phase].reset_index(drop=True)

    # to get dataset between given inning_1_total numbers:

    df = df[(df.inn1_total >= a) & (df.inn1_total <= b)].reset_index(drop=True)
    
    # to get dataset for given innings:
    
    if inning == 1:
        df = df[df.innings == 1].reset_index(drop=True)
    elif inning == 2:
        df = df[df.innings == 2].reset_index(drop=True)
    elif inning == 0:
        df = df
    else:
        return print('Type Correct Inning.')
    
    # selecting striker:
    
    if striker == 0:
        df = df
    else:
        df = df[df.striker == striker]
    
    # fetching batters data
    
    df['wides'] = df['wides'].fillna(0)
    df['noballs'] = df['noballs'].fillna(0)
    df['byes'] = df['byes'].fillna(0)
    
    if ('legbyes') in df.columns:
        df['legbyes'] = df['legbyes'].fillna(0)
    elif ('leg_byes') in df.columns:
        df['legbyes'] = df['leg_byes'].fillna(0)
    else:
        pass
    
    if  ('legal_ball','legal_ball_bowler') not in df.columns:
        df['legal_ball'] = df.apply(lambda x : 0 if x['wides']>0 else 1, axis=1)
        df['legal_ball_bowler'] = df.apply(lambda x : 0 if ((x['wides']>0) | (x['noballs']>0)) else 1, axis=1)
    else:
        pass

    df['dots'] = df.runs_off_bat.apply(lambda x : 1 if x==0 else 0)
    df['ones'] = df.runs_off_bat.apply(lambda x : 1 if x==1 else 0)
    df['twos'] = df.runs_off_bat.apply(lambda x : 1 if x==2 else 0)
    df['threes'] = df.runs_off_bat.apply(lambda x : 1 if x==3 else 0)
    df['fours'] = df.runs_off_bat.apply(lambda x : 1 if x==4 else 0)
    df['sixes'] = df.runs_off_bat.apply(lambda x : 1 if x==6 else 0)

    Runs = pd.DataFrame(df.groupby(['match_id','striker'])['runs_off_bat'].sum()).reset_index().rename(columns={'runs_off_bat':'runs'})
    Balls = pd.DataFrame(df.groupby(['match_id','striker'])['legal_ball'].sum()).reset_index().rename(columns={'legal_ball':'balls'})
    Dots = pd.DataFrame(df.groupby(['match_id','striker'])['dots'].sum()).reset_index()
    Ones = pd.DataFrame(df.groupby(['match_id','striker'])['ones'].sum()).reset_index()
    Twos = pd.DataFrame(df.groupby(['match_id','striker'])['twos'].sum()).reset_index()
    Threes = pd.DataFrame(df.groupby(['match_id','striker'])['threes'].sum()).reset_index()
    Fours = pd.DataFrame(df.groupby(['match_id','striker'])['fours'].sum()).reset_index()
    Sixes = pd.DataFrame(df.groupby(['match_id','striker'])['sixes'].sum()).reset_index()

    playerDf1 = pd.merge(Runs,Balls,on=['match_id','striker'])
    playerDf2 = pd.merge(playerDf1,Fours,on=['match_id','striker'])
    playerDf3 = pd.merge(playerDf2,Sixes,on=['match_id','striker'])
    playerDf4 = pd.merge(playerDf3,Ones,on=['match_id','striker'])
    playerDf5 = pd.merge(playerDf4,Twos,on=['match_id','striker'])
    playerDf6 = pd.merge(playerDf5,Threes,on=['match_id','striker'])
    df = pd.merge(playerDf6,Dots,on=['match_id','striker'])

    df['30s'] = df.apply(lambda x : 1 if x['runs'] >= 30 else 0, axis=1)
    df['50s'] = df.apply(lambda x : 1 if x['runs'] >= 50 else 0, axis=1)
    df['100s'] = df.apply(lambda x : 1 if x['runs'] >= 100 else 0, axis=1)

    Innings = pd.DataFrame(df.groupby(df['striker'])['match_id'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns={'match_id':'innings'})
    Runs = pd.DataFrame(df.groupby(['striker'])['runs'].sum()).reset_index()
    Balls = pd.DataFrame(df.groupby(['striker'])['balls'].sum()).reset_index()
    Fours = pd.DataFrame(df.groupby(['striker'])['fours'].sum()).reset_index()
    Sixes = pd.DataFrame(df.groupby(['striker'])['sixes'].sum()).reset_index()
    n30 = pd.DataFrame(df.groupby(['striker'])['30s'].sum()).reset_index()
    n50 = pd.DataFrame(df.groupby(['striker'])['50s'].sum()).reset_index()
    n100 = pd.DataFrame(df.groupby(['striker'])['100s'].sum()).reset_index()
    Ones = pd.DataFrame(df.groupby(['striker'])['ones'].sum()).reset_index()
    Twos = pd.DataFrame(df.groupby(['striker'])['twos'].sum()).reset_index()
    Threes = pd.DataFrame(df.groupby(['striker'])['threes'].sum()).reset_index()
    Dots = pd.DataFrame(df.groupby(['striker'])['dots'].sum()).reset_index()

    df = pd.merge(Innings,Runs,on='striker').merge(Balls,on='striker').merge(Fours,on='striker').merge(Sixes,on='striker').merge(n30,on='striker').merge(n50,on='striker').merge(n100,on='striker').merge(Ones,on='striker').merge(Twos,on='striker').merge(Threes,on='striker').merge(Dots,on='striker')

    df['RPI'] = df.apply(lambda x : x['runs']/x['innings'], axis=1)
    df['SR'] = df.apply(lambda x : (x['runs']/x['balls'])*100, axis=1)
    df['BPB'] = df.apply(lambda x : BPB(x['balls'],(x['fours']+x['sixes'])), axis=1)
    
    return df

# BOWLING FUNCTION

In [61]:
def Bowler_data_fetching(df, team, opposition, phase, a, b, inning, bowler):
    
    # to get required team:
    
    if team == 0:
        df = df
    elif team == recent_teams:
        df = df[df.bowling_team.isin(recent_teams)].reset_index(drop=True)
    else:
        df = df[df.bowling_team == team].reset_index(drop=True)
        
    # to get required opposition:
    
    if opposition == 0:
        df = df
    elif opposition == recent_teams:
        df = df[df.batting_team.isin(recent_teams)].reset_index(drop=True)
    else:
        df = df[df.batting_team == opposition].reset_index(drop=True)
        
    # to get dataset between given phase:
   
    if phase == 0:
        df = df
    else:        
        df = df[df.phase == phase].reset_index(drop=True)
    
    # to get dataset between given inning_1_total numbers:

    df = df[(df.inn1_total >= a) & (df.inn1_total <= b)].reset_index(drop=True)
    
    # to get dataset for given innings:
    
    if inning == 1:
        df = df[df.innings == 1].reset_index(drop=True)
    elif inning == 2:
        df = df[df.innings == 2].reset_index(drop=True)
    elif inning == 0:
        df = df
    else:
        return print('Type Correct Inning.')
    
    # selecting bowler:
    
    if bowler == 0:
        df = df
    else:
        df = df[df.bowler == bowler]
    
    # fetching batters data
    
    df['wides'] = df['wides'].fillna(0)
    df['noballs'] = df['noballs'].fillna(0)
    df['byes'] = df['byes'].fillna(0)
    
    if ('legbyes') in df.columns:
        df['legbyes'] = df['legbyes'].fillna(0)
    elif ('leg_byes') in df.columns:
        df['legbyes'] = df['leg_byes'].fillna(0)
    else:
        pass

    if  ('legal_ball','legal_ball_bowler') not in df.columns:
        df['legal_ball'] = df.apply(lambda x : 0 if x['wides']>0 else 1, axis=1)
        df['legal_ball_bowler'] = df.apply(lambda x : 0 if ((x['wides']>0) | (x['noballs']>0)) else 1, axis=1)
    else:
        pass

    df['dots'] = df.runs_off_bat.apply(lambda x : 1 if x==0 else 0)
    df['ones'] = df.runs_off_bat.apply(lambda x : 1 if x==1 else 0)
    df['twos'] = df.runs_off_bat.apply(lambda x : 1 if x==2 else 0)
    df['threes'] = df.runs_off_bat.apply(lambda x : 1 if x==3 else 0)
    df['fours'] = df.runs_off_bat.apply(lambda x : 1 if x==4 else 0)
    df['sixes'] = df.runs_off_bat.apply(lambda x : 1 if x==6 else 0)

    df['isBowlerWicket'] = df.apply(lambda x : Bowler_WK(x['player_dismissed'],x['wicket_type']),axis=1)
    df['runs'] = df['runs_off_bat'].astype(float)
    df['wide'] = df['wides'].fillna(0)
    df['noball'] = df['noballs'].fillna(0)
    df['isBowlerRuns'] = df['runs'] + df['wide'] + df['noball']

    wickets = pd.DataFrame(df.groupby(['match_id','bowler'])['isBowlerWicket'].sum()).reset_index().rename(columns={'isBowlerWicket':'wickets'})
    balls = pd.DataFrame(df.groupby(['match_id','bowler'])['legal_ball_bowler'].sum()).reset_index().rename(columns={'legal_ball_bowler':'balls'})
    runs = pd.DataFrame(df.groupby(['match_id','bowler'])['isBowlerRuns'].sum()).reset_index().rename(columns={'isBowlerRuns':'runs'})
    fours = pd.DataFrame(df.groupby(['match_id','bowler'])['fours'].sum()).reset_index().rename(columns={'isFour':'fours'})
    sixes = pd.DataFrame(df.groupby(['match_id','bowler'])['sixes'].sum()).reset_index().rename(columns={'isSix':'sixes'})

    df = pd.merge(wickets,balls,on=['match_id','bowler']).merge(runs,on=['match_id','bowler']).merge(fours,on=['match_id','bowler']).merge(sixes,on=['match_id','bowler'])

    df['runs'] = df.runs.astype('int')
    df['3_w_b'] = df.apply(lambda x : 4 if x['wickets'] == 3 else 0, axis=1)
    df['4_w_b'] = df.apply(lambda x : 8 if x['wickets'] == 4 else 0, axis=1)
    df['5_w_b'] = df.apply(lambda x : 16 if x['wickets'] == 5 else 0, axis=1)

    innings = pd.DataFrame(df.groupby(['bowler'])['match_id'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns={'match_id':'innings'})
    Runs = pd.DataFrame(df.groupby(['bowler'])['runs'].sum()).reset_index()
    Balls = pd.DataFrame(df.groupby(['bowler'])['balls'].sum()).reset_index()
    Wickets = pd.DataFrame(df.groupby(['bowler'])['wickets'].sum()).reset_index()
    w3b = pd.DataFrame(df.groupby(['bowler'])['3_w_b'].sum()).reset_index()
    w4b = pd.DataFrame(df.groupby(['bowler'])['4_w_b'].sum()).reset_index()
    w5b = pd.DataFrame(df.groupby(['bowler'])['5_w_b'].sum()).reset_index()

    df = pd.merge(innings,Wickets,on='bowler').merge(Runs,on='bowler').merge(Balls,on='bowler').merge(w3b,on='bowler').merge(w4b,on='bowler').merge(w5b,on='bowler')
    
    df['ECO'] = (df['runs']/(df['balls']/6))
    df['SR'] = df.apply(lambda x : x['runs'] if x['wickets']==0 else (x['runs']/x['wickets']), axis=1)
    df['AVG'] = df.apply(lambda x : x['balls'] if x['wickets']==0 else (x['balls']/x['wickets']), axis=1)
    
    return df

# DATA CREATED FOR BATTERS D: 18/08/2023

In [8]:
# batter_pp = Batter_data_fetching(df, 180, 250, 3, 'Powerplay')

# openers = ['RD Gaikwad','YBK Jaiswal','Shubman Gill', 'Ishan Kishan']

# openers_per = batter_pp[batter_pp.striker.isin(openers)].reset_index(drop=True)

# plt.figure(figsize=(6,4))
# plt.rcParams['axes.facecolor'] = '#fcffe9'#'#caf4f4'#'#caefd1''#fde0d9''#fcffe9'
# plt.scatter(openers_per.BPB,openers_per.SR,label='Indian T20I Openers')
# for i in range(len(openers_per)):
#     if openers_per.striker[i] == 'RD Gaikwad':
#         plt.text(openers_per.BPB[i]-0.43,openers_per.SR[i]-0.5,openers_per.striker[i])
#     else:
#         plt.text(openers_per.BPB[i]+0.04,openers_per.SR[i]-0.5,openers_per.striker[i])
# plt.title('Performance in Powerplay in the IPL, When Inning score is >180', fontsize=10)
# plt.xlabel('Balls per Boundary')
# plt.ylabel('Strike Rate')
# plt.legend()
# # plt.savefig('Images/openers.jpg', dpi=1200)
# plt.show()

# batter_mo = Batter_data_fetching(df, 180, 250, 3, 'Middle')
# middlers = ['S Dube','SV Samson','JM Sharma','RK Singh','Tilak Varma','Washington Sundar','Shahbaz Ahmed']
# middle_per = batter_mo[batter_mo.striker.isin(middlers)].reset_index(drop=True)

# plt.figure(figsize=(6,4))
# plt.rcParams['axes.facecolor'] = '#fcffe9'#'#caf4f4'#'#caefd1''#fde0d9''#fcffe9'
# plt.scatter(middle_per.BPB,middle_per.SR,label='Indian T20I Batter')
# for i in range(len(middle_per)):
#     if middle_per.striker[i] == 'Washington Sundar':
#         plt.text(middle_per.BPB[i]-2.6,middle_per.SR[i]-1,middle_per.striker[i])
#     elif middle_per.striker[i] == 'RK Singh':
#         plt.text(middle_per.BPB[i]-1.3,middle_per.SR[i]-1,middle_per.striker[i])
#     else:
#         plt.text(middle_per.BPB[i]+0.15,middle_per.SR[i]-1,middle_per.striker[i])
# plt.title('Performance in Middle Overs in the IPL, When Inning score is >180', fontsize=10)
# plt.xlabel('Balls per Boundary')
# plt.ylabel('Strike Rate')
# plt.legend()
# # plt.savefig('Images/middle.jpg', dpi=1200)
# plt.show()

# batter_do = Batter_data_fetching(df, 180, 250, 3, 'Death')

# deathers = ['S Dube','SV Samson','JM Sharma','RK Singh','Tilak Varma','Washington Sundar','Shahbaz Ahmed']

# death_per = batter_do[batter_do.striker.isin(deathers)].reset_index(drop=True)

# plt.figure(figsize=(6,4))
# plt.rcParams['axes.facecolor'] = '#fcffe9'#'#caf4f4'#'#caefd1''#fde0d9''#fcffe9'
# plt.scatter(death_per.BPB,death_per.SR,label='Indian T20I Batter')
# for i in range(len(death_per)):
#     if death_per.striker[i] == 'Washington Sundar':
#         plt.text(death_per.BPB[i],death_per.SR[i]+2,death_per.striker[i])
#     elif death_per.striker[i] == 'RK Singh':
#         plt.text(death_per.BPB[i],death_per.SR[i]+2,death_per.striker[i])
#     elif death_per.striker[i] == 'JM Sharma':
#         plt.text(death_per.BPB[i]-0.6,death_per.SR[i]-5,death_per.striker[i])
#     elif death_per.striker[i] == 'S Dube':
#         plt.text(death_per.BPB[i]+0.1,death_per.SR[i]-1,death_per.striker[i])
#     elif death_per.striker[i] == 'Shahbaz Ahmed':
#         plt.text(death_per.BPB[i]-1.6,death_per.SR[i]-1,death_per.striker[i])
#     else:
#         plt.text(death_per.BPB[i]+0.06,death_per.SR[i]-1,death_per.striker[i])
# plt.title('Performance in Death Overs in the IPL, When Inning score is >180', fontsize=10)
# plt.xlabel('Balls per Boundary')
# plt.ylabel('Strike Rate')
# # plt.savefig('Images/death.jpg', dpi=1200)
# plt.legend()
# plt.show()

# DATA CREATED FOR BOWLERS

In [9]:
# bowler_pp = Bowler_data_fetching(df, 180, 240, 3, 'Powerplay')

# pp = ['JJ Bumrah','Mukesh Kumar','Washington Sundar','Arshdeep Singh','M Prasidh Krishna','Avesh Khan']

# bowl_pp = bowler_pp[bowler_pp.bowler.isin(pp)].reset_index(drop=True)

# plt.figure(figsize=(6,4))
# plt.rcParams['axes.facecolor'] = '#fcffe9'#'#caf4f4'#'#caefd1''#fde0d9''#fcffe9'
# plt.scatter(bowl_pp.ECO,bowl_pp.AVG,label='Indian T20I Bowler')
# for i in range(len(bowl_pp)):
#     if bowl_pp.bowler[i] == 'Mukesh Kumar':
#         plt.text(bowl_pp.ECO[i]-0.17,bowl_pp.AVG[i]+1,bowl_pp.bowler[i])
#     elif bowl_pp.bowler[i] == 'Avesh Khan':
#         plt.text(bowl_pp.ECO[i]-0.6 ,bowl_pp.AVG[i]-2.5 ,bowl_pp.bowler[i])
#     elif bowl_pp.bowler[i] == 'Arshdeep Singh':
#         plt.text(bowl_pp.ECO[i]-0.9 ,bowl_pp.AVG[i]-0.6 ,bowl_pp.bowler[i])
#     else:
#         plt.text(bowl_pp.ECO[i]+0.06 ,bowl_pp.AVG[i]-0.6 ,bowl_pp.bowler[i])
# plt.title('Performance in Powerplay in the IPL, When Inning score is >180', fontsize=12)
# plt.xlabel('Economy')
# plt.ylabel('Avarage')
# plt.legend()
# # plt.savefig('Images/pppbowler.jpg', dpi=1200)
# plt.show()

# bowler_mo = Bowler_data_fetching(df, 180, 240, 3, 'Middle')

# mo = ['JJ Bumrah','Mukesh Kumar','Ravi Bishnoi','Washington Sundar','S Dube','Arshdeep Singh','M Prasidh Krishna','Shahbaz Ahmed','Avesh Khan']

# bowl_mo = bowler_mo[bowler_mo.bowler.isin(mo)].reset_index(drop=True)

# plt.figure(figsize=(6,4))
# plt.rcParams['axes.facecolor'] = '#fcffe9'#'#caf4f4'#'#caefd1''#fde0d9''#fcffe9'
# plt.scatter(bowl_mo.ECO,bowl_mo.AVG,label='Indian T20I Bowler')
# for i in range(len(bowl_mo)):
#     if bowl_mo.bowler[i] == 'Arshdeep Singh':
#         plt.text(bowl_mo.ECO[i]-1.6 ,bowl_mo.AVG[i]-0.8,bowl_mo.bowler[i])
#     elif bowl_mo.bowler[i] == 'Mukesh Kumar':
#         plt.text(bowl_mo.ECO[i]-1.3 ,bowl_mo.AVG[i]+1.3,bowl_mo.bowler[i])
#     else:
#         plt.text(bowl_mo.ECO[i]+0.09 ,bowl_mo.AVG[i]-0.8 ,bowl_mo.bowler[i])
# plt.title('Performance in Middle Overs in the IPL, When Inning score is >180', fontsize=10)
# plt.xlabel('Economy')
# plt.ylabel('Avarage')
# plt.legend()
# # plt.savefig('Images/moobowler.jpg', dpi=1200)
# plt.show()

# bowler_do = Bowler_data_fetching(df, 180, 240, 3, 'Death')

# do = ['JJ Bumrah','Mukesh Kumar','Ravi Bishnoi','Washington Sundar','S Dube','Arshdeep Singh','M Prasidh Krishna','Shahbaz Ahmed','Avesh Khan']

# bowl_do = bowler_do[bowler_do.bowler.isin(do)].reset_index(drop=True)

# plt.figure(figsize=(6,4))
# plt.rcParams['axes.facecolor'] = '#fcffe9'#'#caf4f4'#'#caefd1''#fde0d9''#fcffe9'
# plt.scatter(bowl_do.ECO,bowl_do.AVG,label='Indian T20I Bowler')
# for i in range(len(bowl_do)):
#     if bowl_do.bowler[i]=='M Prasidh Krishna':
#         plt.text(bowl_do.ECO[i]-3.25 ,bowl_do.AVG[i]-0.5 ,bowl_do.bowler[i])
#     elif bowl_do.bowler[i]=='Shahbaz Ahmed':
#         plt.text(bowl_do.ECO[i]-3 ,bowl_do.AVG[i]-0.5,bowl_do.bowler[i])
#     elif bowl_do.bowler[i]=='JJ Bumrah':
#         plt.text(bowl_do.ECO[i] ,bowl_do.AVG[i]-1.5,bowl_do.bowler[i])
#     else:
#         plt.text(bowl_do.ECO[i]+0.14 ,bowl_do.AVG[i]-0.5 ,bowl_do.bowler[i])
# plt.title('Performance in Death Overs in the IPL, When Inning score is >180', fontsize=10)
# plt.xlabel('Economy')
# plt.ylabel('Avarage')
# plt.legend()
# # plt.savefig('Images/doobowler.jpg', dpi=1200)
# plt.show()

# EXAMPLES OF DATA FETCHING USING FUNCTIONS:

In [76]:
Batter_data_fetching(df, 'Royal Challengers Bangalore', 0, 0, 180,200, 2, 'V Kohli' ) #Batter_data_fetching(df, team, opposition, phase, a, b, inning, striker)

,striker,innings,runs,balls,fours,sixes,30s,50s,100s,ones,twos,threes,dots,RPI,SR,BPB
0,V Kohli,29,960,693,91,28,15,7,2,322,53,0,215,33.103448,138.528139,5.823529


In [77]:
Bowler_data_fetching(df, 'Sunrisers Hyderabad', 0, 'Death', 0, 260, 2, 'B Kumar') #Bowler_data_fetching(df, team, opposition, phase, a, b, inning, bowler):

,bowler,innings,wickets,runs,balls,3_w_b,4_w_b,5_w_b,ECO,SR,AVG
0,B Kumar,57,34,713,493,12,0,0,8.677485,20.970588,14.5
